In [1]:
# valuation_date時におけるcodeで指定した銘柄の個別銘柄分析
# SBI証券のモバイルの銘柄分析を参考に作成

In [2]:
### パラメータ設定セル
from datetime import date
from dateutil.relativedelta import relativedelta

# パラメータをセット
# code: 証券コード
# valuation_date: 過去にさかのぼって評価日をdatetime.dateでセット可能。デフォルト値は今日(date.today())
code = 1899
valuation_date = date.today()
valuation_date = date(2024, 12, 16)


In [3]:
# 読み込みファイルパスの設定とimportしたいmoduleパス(pythonパス)の設定
from pathlib import Path
import os, sys

CURRENT_DIR = Path(os.getcwd())
PJ_DIR = CURRENT_DIR.parent.parent
LIB_DIR = PJ_DIR / "lib"
DATA_DIR = PJ_DIR / "data" 

sys.path.append(str(LIB_DIR))

# notebook内で利用するmoduleのimport
from lib_dataprocess import PricelistPl, FinancequotePl, KessanPl, KessanFig, PricelistFig, read_data, get_df_latest_yearly_performance, get_latest_stockprice, print_finance_quote
from lib_dataprocess import get_fig_expected_performance_progress_rate_pycharts, MeigaralistPl, get_fig_actual_performance_progress_rate_pycharts, ShikihoPl
import polars as pl

# dataの読み込み
fp = DATA_DIR/"raw_pricelist.parquet"
df = read_data(fp)
PPL = PricelistPl(df)

# dataの読み込み
fp = DATA_DIR/"reviced_pricelist.parquet"
df = read_data(fp)
RevPPL = PricelistPl(df)

fp = DATA_DIR/"finance_quote.parquet"
df = read_data(fp)
FPL = FinancequotePl(df)
FPL.with_columns_market_cap()

fp = DATA_DIR/"kessan.parquet"
df = read_data(fp)
KPL = KessanPl(df)

fp = DATA_DIR/"meigaralist.parquet"
df = read_data(fp)
MPL = MeigaralistPl(df)

fp = DATA_DIR/"shikiho.parquet"
df = read_data(fp)
SPL = ShikihoPl(df)

In [4]:
### 最新情報
# 指定したcodeの指定した日における株価と各種ファンダメンタルズデータをまとめて標準出力する
# pricelist_dfは、raw_pricelistかreviced_pricelistかケースに応じて使い分ける。
# def print_finance_quote(
#        pricelist_df: pl.DataFrame,
#        finance_quote_df: pl.DataFrame,
#        code: int, 
#        valuation_date: date=date.today()
#    ) -> None:
print_finance_quote(PPL.df, FPL.df, code, valuation_date=valuation_date)
print()
SPL.print_stock_df(code, num=4, valuation_date=valuation_date)

福田組(1899)の銘柄情報

終値: 5200.0円(2024年12月16日)
予想配当利回り: 2.12%(2024年12月16日)
予想PER: 11.03倍(2024年12月16日)
実績PBR: 0.52倍(2024年12月16日)
自己資本比率: 58.2%(2024年12月16日)
予想ROE: 4.71%(2024年12月16日)
予想ROA: 2.74%(2024年12月16日)
時価総額: 467.4億円(2024年12月16日)

1899(福田組)の四季報データ履歴

発行日: 2024年09月13日
【好転】
  大規模商業施設完工、大型不動産物件売却で上期大幅増益。民間建築軸に受注増。下期は採算改善工事増加。人件費増でも営業益回復。25年12月期は大都市再開発工事、工場が進捗。不動産事業反動減。
【受注】
  関東で大型商業施設、新潟で工場、複数のマンションなど人材確保課題も受注順調。環境視点のリニューアル工事や太陽光発電など再生可能エネ関連に注力。

発行日: 2024年06月14日
【増額】
  大規模商業施設や再開発工事などが牽引。トンネル工事も寄与。大型低採算工事なくなり、価格転嫁浸透で建設事業の採算改善。原材料高、人件費増だが、上期の大型不動産物件売却で営業益上振れ。
【不動産】
  物流団地などの造成事業を首都圏・九州で展開。好採算事業で開拓に注力。首都圏・新潟などで大規模複合施設、商業施設、マンション工事が順調に進捗。

発行日: 2024年03月16日
【好転】
  期初単体繰越高1219億円(前年比7%増)。大型商業施設や地域再開発工事など進捗。トンネル・浄水場も寄与。リニューアルは着実。低採算工事影響薄れる。人件費増・経費増でも営業益好転。
【大規模】
  建築ではショッピングセンター、複合再開発案件など大規模案件を相次ぎ受注。グループ会社は小回り生かしリニューアル小型短期案件の受注を積極化。

発行日: 2023年12月15日
【好転】
  商業施設完工。大型複合開発始動し、トンネル貢献も、低採算工事の発生響き前号比営業益下振れ。24年12月期は建築が商業・物流の需要増。大型継続工事、更新案件寄与。低採算工事減少し営業益好転。
【受注】
  首都圏、新潟・東北で大型商業施設、複数の物流倉庫を受注。西日本でもマンション

In [5]:
### 業績推移

# KessanFig(
#    code: int, 
#    settlement_type: Literal["通期", "四半期"], 
#    output_target: str = "jupyter",
#    start_settlement_date: date = date(1900, 1, 1),
#    end_settlement_date: date = date(2999, 12, 31)
#)

KFIG = KessanFig(code, "通期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()

In [6]:
# 今季決算予想発表の推移
df = KPL.get_settlement_forcast(code, valuation_date)
df

2024-03-31


code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
1899,2024-12-31,"""予""",2024-02-14,166400,5800,6000,3900,471.4,110.0,4


In [7]:
### 通期業績、業績予想
# def get_df_latest_yearly_performance(code: int, valuation_date: date=date.today()) -> pl.DataFrame:
df = get_df_latest_yearly_performance(code, valuation_date)
df

福田組(1899)の通期決算(評価日：2024年12月16日)


決算期,決算発表日,売上高,営業利益,経常利益,純利益,EPS,1株配当
str,date,i64,i64,i64,i64,f64,f64
"""2024年12月期(予)""",2024-02-14,166400,5800,6000,3900,471.4,110.0
"""2023年12月期""",2024-02-14,162243,5205,5478,3386,400.1,110.0


In [8]:
KFIG = KessanFig(code, "四半期", end_settlement_date=valuation_date)
KFIG.add_trace_profits()
KFIG.fig.show()
KPL.get_latest_quater_settlement(code, valuation_date)

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
1899,2024-09-30,"""四""",2024-11-08,40884,2205,2247,1488,179.9,5.4,3
1899,2024-06-30,"""四""",2024-08-08,34299,871,957,577,69.7,2.5,2
1899,2024-03-31,"""四""",2024-05-10,41419,2659,2751,1883,227.7,6.4,1
1899,2023-12-31,"""四""",2024-02-14,50267,1844,1926,1160,137.1,3.7,4
1899,2023-09-30,"""四""",2023-11-10,42086,1826,1849,1140,134.4,4.3,3
1899,2023-06-30,"""四""",2023-08-10,33988,207,284,132,15.6,0.6,2
1899,2023-03-31,"""四""",2023-05-11,35902,1328,1419,954,112.3,3.7,1
1899,2022-12-31,"""四""",2023-02-14,46252,207,271,120,14.1,0.4,4


In [9]:
# codeで指定した銘柄のevaluation_dateで指定した時点での最新の年度決算予想に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# from plotly.graph_objects import Figure
# def get_fig_expected_performance_progress_rate_pycharts(code: int, evaluation_date: date=date.today()) -> Figure:

fig1 = get_fig_expected_performance_progress_rate_pycharts(code, valuation_date)
fig1.show()

福田組(1899)の2024年12月期第3四半期決算進捗率(評価日：2024-12-16)


In [10]:
last_year_valuation_date = date(valuation_date.year-1, valuation_date.month, valuation_date.day)

# codeで指定した銘柄のevaluation_dateで指定した時点での最新の四半期決算発表に基づく
# 売上高~純利益の決算進捗率を円グラフで表示するためのfigを返す
# valuation_dateで指定する日において、当年度の本決算が発表されていない日を指定した場合は前期末第4四半期の進捗率を表示するので、進捗率100%で表示される。
fig2 = get_fig_actual_performance_progress_rate_pycharts(code, last_year_valuation_date, KPL.df, MPL.df)
fig2.show()

福田組(1899)の2023年12月期第3四半期決算進捗率(評価日：2023-12-16)


In [11]:
#### valuation_dateに過去日を指定した場合、valuation_dateを含む期の株価推移
#### 答え合わせ

In [12]:
# 株価推移
# 前期算発表翌営業日～今期決算発表日翌営業日
start_date, end_date = KPL.get_current_settlement_period_by_announcement_date(code, valuation_date, "四")

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>start_date)
start_date = df["date"].to_list()[0]

df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")>end_date)
if not end_date == date(2999, 12, 31):
    end_date = df["date"].to_list()[0]

PFIG = PricelistFig(code, RevPPL.df, MPL.df, start_date, end_date)
df = RevPPL.df.filter(pl.col("code")==code).filter(pl.col("date")<=valuation_date)
vline_date = df["date"].to_list()[-1]
PFIG.add_vline(vline_date)
PFIG.fig.show()

In [13]:
# appendix
KPL.df.filter(pl.col("code")==code).filter(pl.col("announcement_date")>=date(2024,1,1))

code,settlement_date,settlement_type,announcement_date,sales,operating_income,ordinary_profit,final_profit,reviced_eps,dividend,quater
i64,date,str,date,i64,i64,i64,i64,f64,f64,i64
1899,2023-12-31,"""四""",2024-02-14,50267,1844,1926,1160,137.1,3.7,4
1899,2023-12-31,"""本""",2024-02-14,162243,5205,5478,3386,400.1,110.0,4
1899,2024-12-31,"""予""",2024-02-14,166400,5800,6000,3900,471.4,110.0,4
1899,2024-03-31,"""四""",2024-05-10,41419,2659,2751,1883,227.7,6.4,1
1899,2024-06-30,"""四""",2024-08-08,34299,871,957,577,69.7,2.5,2
1899,2024-09-30,"""四""",2024-11-08,40884,2205,2247,1488,179.9,5.4,3


In [14]:
PFIG = PricelistFig(code, RevPPL.df, MPL.df, date(2023, 12, 1), date.today())
PFIG.fig.show()

In [15]:
from lib_dataprocess import CreditbalancePl

CPL = CreditbalancePl()
CPL.with_columns_margin_ratio()
CPL.df.filter(pl.col("code")==code)

code,date,unsold_margin,purchase_margin,margin_ratio
i64,date,i64,i64,f64
1899,2008-12-30,207000,610000,2.95
1899,2009-01-09,229000,630000,2.75
1899,2009-01-16,290000,628000,2.17
1899,2009-01-23,236000,621000,2.63
1899,2009-01-30,226000,615000,2.72
…,…,…,…,…
1899,2024-11-15,7000,27300,3.9
1899,2024-11-22,9800,28200,2.88
1899,2024-11-29,20200,31100,1.54
